# **[Tensorflow] SKIP_GRAM을 이용한 word2vec**

### **1. Data loading**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from konlpy.tag import Komoran

In [ ]:
train_df = pd.read_csv('./input/ratings_train.txt', delimiter='\t')
pos_df = train_df[train_df['label'] == 1]['document']
neg_df = train_df[train_df['label'] == 0]['document']

### **2. Data preprocessing**
#### **(1) Text를 word의 seq로 변경**

In [ ]:
komoran = Komoran() 

In [ ]:
pos_data = []
pos_label = []
for pos in pos_df:
    try:
        words = komoran.nouns(pos)
        # 단어가 3개 이상 등장한 review 에 대해서만 데이터 set을 생성한다.
        if len(words) > 3:
            pos_data.append(words)
            # 긍정을 의미하는 label 이다.
            pos_label.append(1)
    except:
        pass

In [ ]:
neg_data = []
neg_label = []
for neg in neg_df:
    try:
        words = komoran.nouns(neg)
        if len(words) > 3:
            neg_data.append(words)
            neg_label.append(0)
    except:
        pass

In [ ]:
texts = pos_data + neg_data
labels = pos_label + neg_label

print(len(texts))
print(len(labels))

In [ ]:
texts[:5]

#### **(2) Word의 seq를 number의 seq로 변경 (word2vec의 input : 정수)**

In [ ]:
# indexing dictionary
words = []
for text in texts:
    words.extend(text)

In [ ]:
from collections import Counter
voca_size = 25000
corpus = {}

# 빈도수가 높은 단어 순서대로, indexing이 된다.
for word, freq in Counter(words).most_common(voca_size):
    corpus[word] = len(corpus)

# 결과 디버깅을 위하여, number seq에서, number에 대응하는 word를 찾기 위한 lookup dictionary
corpus_rev = dict(zip(corpus.values(), corpus.keys()))

In [ ]:
# Number seq를 생성한다. 즉 단어에 대응되는 num의 seq로 바꾸는 작업
num_seqs = []

for text in texts:
    num_seq = []
    for word in text:
        if word in corpus:
            idx = corpus[word]
        else:
            idx = 0  # 없는 단어는 0으로 한다. voca_size를 지정했기 때문이다.
        num_seq.append(idx)
    num_seqs.append(num_seq)

In [ ]:
num_seqs[0]

#### **(3) Input batch (target word, source context word)의 tuple를 생성**
 - SKIP-GRAM는 target word를 이용하여 source context word를 예측하는 학습과정을 가진다.
 - Source context word left, target word, source context right 이렇게 주어지면, 학습 모델은 target word -> source context left, target word -> source context right가 된다.

In [ ]:
# Seqs : [ src_ctx_left, target, src_ctx_right ]
# Inputs : [ (target, src_ctx_left), (target, src_ctx_right) ]
# Batch : [ target ], label : [ src_ctx_left ]
#         [ target ],         [ src_ctx_right ]
def generate_batch_data(num_seqs, batch_size, window_size):
    # Batch 데이터
    batch_data = []
    label_data = []
    
    while len(batch_data) < batch_size:
        # Random하게 review data 1개를 뽑는다.
        random_seq = np.random.choice(num_seqs)
        span = skip_window * 2 + 1
        
        windows_seq = []
        for i in range(len(random_seq) - span + 1):
            windows_seq.append(random_seq[i:i+span])
            
        # Target index는 가운데에 있는 것이 될 것이다.
        target_indices = [span//2 for _ in range(len(windows_seq))]
        
        # Target word, source context word list의 tuple을 생성
        target_and_sources = [(seq[idx], seq[:idx] + seq[idx+1:]) 
                              for idx, seq in zip(target_indices, windows_seq)]

        # Target word, source context word의 tuple을 생성
        target_and_source = [(target, source) 
                             for target, sources in target_and_sources 
                             for source in sources]
        
        # Target word list(input), source context word list(output)로 나눈다.
        target_word_list, src_ctx_word_list = [list(elem) for elem in zip(*target_and_source)]
        batch_data.extend(target_word_list)
        label_data.extend(src_ctx_word_list)
                
    # 마지막 iteration에서 batch size를 초과할 수 있으므로, batch 크기만 생성
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Placeholder에 feeding하기 위해서는 numpy array 형태로 변환한다.
    batch_data = np.array(batch_data)
    # Nce_loss에서 사용하기 위해서 shape을 변경한다.
    label_data = np.transpose(np.array([label_data]))
    
    return (batch_data, label_data)

In [ ]:
# Validation set을 생성한다. 생성한 embedding matrix와의 거리를 측정할 수 있다.
# 이것은 단순히 similarity를 보여주기 위한 예이다.
valid_words = ['한국', '사랑', '스릴러', '장르']
valid_words = [corpus[elem] for elem in valid_words]
valid_datasets = tf.constant(valid_words, dtype=tf.int32)

### **3. modeling using tensorflow**

In [ ]:
embedding_size = 128
batch_size = 128
skip_window = 1  # How many subnets to consider left and right
num_sampled = 64  # Number of negative examples to sample
learning_rate = 0.001
num_steps = 1000001  # Tensorflow tutorial

# Input data
with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

"""
Embedding -> Train input -> embed
[[1,2,3], -> [0,2]       -> [[1,2,3],
 [4,5,6],                    [7,8,9]]
 [7,8,9]]
"""
with tf.device('/GPU:0'):
    with tf.name_scope('embeddings'):
        embeddings = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1, 1))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Weights
    with tf.name_scope('weights'):
        nce_weights = tf.Variable(
            tf.truncated_normal([voca_size, embedding_size], 
                                stddev = 1.0 / math.sqrt(embedding_size)))
    # Biases
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([voca_size]))

# Loss
with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=train_labels,
            inputs=embed,
            num_sampled=num_sampled,
            num_classes=voca_size))

# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
# 검증 단어 집합과 임베딩된 모든 단어 사이의 코사인 유사도를 구해보자.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
# 검증 단어 집합의 embedding을 구한다.
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_datasets)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
# session 초기화
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
num_steps = 50000
for i in range(num_steps):
    batch_inputs, batch_labels = generate_batch_data(num_seqs, batch_size, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
    sess.run(optimizer, feed_dict=feed_dict)
    if i % 100 == 0:
        print("=================================")
        loss_val = sess.run(loss, feed_dict=feed_dict)
        # Githup에 올릴때는 주석을 한다. 결과를 보고 싶으면 주석을 해제하면 된다.
        print('Loss at step {}: {}'.format(i, loss_val))
        
        # Valid word와의 similarity를 확인하여 검증을 해보자.
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = corpus_rev[valid_words[j]]
            top_k = 5
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = 'Nearest to {}:'.format(valid_word)
            for k in range(top_k):
                close_word = corpus_rev[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            # Githup에 올릴때 주석, 결과를 보고 싶으면 주석을 해제하면 된다.
            print(log_str)

In [ ]:
sess.close()